In [2]:
pip install pendulum

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 2.9 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.5 MB 2.4 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/11.5 MB 2.2 MB/s eta 0:00:05
   -------- ------------------------------- 2.4/11.5 MB 2.5 MB/s eta 0:00:04
   --------- ------------------------------ 2.6/11.5 MB 2.5 MB/s eta 0:00:04
   ----------- ---------------------------- 3.4/11.5 MB 2.5 MB/s eta 0:00:04
   -------------- ------------------------- 4.2/11.5 MB 2.6 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.5 MB 2.6 MB/s eta 0:00:03
   ------------------- -------------------- 5.5/11.5 MB 2.7 MB/s eta 0:00:03
   --------------------- ------------------ 6.0/11.5 MB 2.8 MB/s eta 0:00:02
   ---------------------- ----------------- 6.6/11.5 MB 2.7 MB/s eta 0:00:02
   ----------

In [11]:
pip install scikit-learn


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.1 MB 1.8 MB/s eta 0:00:06
   ----- ---------------------------------- 1.6/11.1 MB 2.9 MB/s eta 0:00:04
   -------- ------------------------------- 2.4/11.1 MB 3.1 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.1 MB 3.1 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.1 MB 3.1 MB/s eta 0:00:03
   ---------------- ----------------------- 4.5/11.1 MB 3.2 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.1 MB 3.2 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.1 MB 3.3 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.1 MB 3.4 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/11.1 MB 3.4 MB/s eta 0:00:02
   ------------------------------ --------- 8.4/11.1 MB 3.4 MB/s eta 0:00:01
   ----------

In [2]:
from iointel import Agent, Workflow
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

# API Anahtarını ayarla
os.environ["OPENAI_API_KEY"] = "io-v2-eyJhbGciOi..."  # Gerçek IO.net API anahtarını buraya gir

# CSV verisini oku
df = pd.read_csv(r"C:\Users\sevva\OneDrive\Masaüstü\gpu_listesi_duzenlenmis.csv")

# Performansı ve kullanım alanını birleştirerek metin oluştur
df["Performans"] = df["Performans"].str.strip().str.lower()
df["etiket_birlesik"] = df.apply(
    lambda r: f"{r['GPU Modeli']} {r['Kullanım Alanları']} {r['Performans']} {r['VRAM (GB)']}GB {r['CUDA Çekirdekleri']} çekirdek {r['Bellek Bant Genişliği (GB/s)']}GB/s", 
    axis=1
)

# NaN değerleri doldur
df.ffill(inplace=True)

# Embedding modeli yükle
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# IO.net Agent oluştur
agent = Agent(
    name="GPU Önerici Agent",
    instructions="Kullanıcının ihtiyaçlarına göre uygun GPU öner. Gerekirse ek bilgi iste.",
    model="meta-llama/Llama-3.3-70B-Instruct",
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://api.intelligence.io.solutions/api/v1"
)

# Yardımcı fonksiyonlar
def normalize(col):
    return (col - col.min()) / (col.max() - col.min())

def sayisal_aralik_belirle(prompt, kriter):
    if kriter == "vram":
        m = re.search(r'vram\s*(\d+)\s*gb', prompt)
        if m:
            return int(m.group(1))
    if kriter == "cuda":
        m = re.search(r'cuda\s*\d+\s*çekirdek', prompt)
        if m:
            return int(re.search(r'(\d+)', m.group()).group(1))
    return None

def performans_belirle(prompt):
    pl = prompt.lower()
    if "çok yüksek" in pl:
        return "çok yüksek"
    if "yüksek" in pl:
        return "yüksek"
    if "orta" in pl:
        return "orta"
    if "düşük" in pl:
        return "düşük"
    return None

def kullanım_alani_belirle(prompt):
    alanlar = ["oyun", "render", "ofis", "ai", "yapay zeka"]
    for a in alanlar:
        if a in prompt.lower():
            return a
    return None

# GPU öneri fonksiyonu
def gpu_oner(prompt, max_results=5):
    p = prompt.lower()
    min_vram = sayisal_aralik_belirle(p, "vram") or 0
    min_cuda = sayisal_aralik_belirle(p, "cuda") or 0
    perf = performans_belirle(p)
    alan = kullanım_alani_belirle(p)

    df_filt = df.copy()
    if min_vram:
        df_filt = df_filt[df_filt["VRAM (GB)"] >= min_vram]
    if min_cuda:
        df_filt = df_filt[df_filt["CUDA Çekirdekleri"] >= min_cuda]
    if perf:
        df_filt = df_filt[df_filt["Performans"] == perf]
    if alan:
        df_filt = df_filt[df_filt["Kullanım Alanları"].str.lower().str.contains(alan)]

    if df_filt.empty:
        return "🤔 Bu kriterlere uygun GPU bulunamadı. Lütfen daha detaylı bilgi verin."

    df_filt.ffill(inplace=True)

    df_filt['n_vram'] = normalize(df_filt['VRAM (GB)'])
    df_filt['n_cuda'] = normalize(df_filt['CUDA Çekirdekleri'])
    df_filt['n_band'] = normalize(df_filt['Bellek Bant Genişliği (GB/s)'])

    text_embeds = embed_model.encode(df_filt['etiket_birlesik'].tolist())
    nums = df_filt[['n_vram','n_cuda','n_band']].values
    combined = np.hstack([text_embeds, nums])
    prompt_embed = embed_model.encode([prompt])[0]
    prompt_ext = np.concatenate([prompt_embed, np.zeros(3)])

    sims = cosine_similarity([prompt_ext], combined)[0]
    idx = np.argsort(sims)[-max_results:][::-1]

    gpu_list = []
    for i in idx:
        gpu = df_filt.iloc[i]
        gpu_list.append(f"{gpu['GPU Modeli']} (VRAM: {gpu['VRAM (GB)']}GB, CUDA: {gpu['CUDA Çekirdekleri']}, Performans: {gpu['Performans']})")

    # Agent ile kayıt
    wf = Workflow(text=prompt, client_mode=False)
    wf.custom(
        name="gpu_oner_task",
        objective=f"Prompt: {prompt} - Önerilen GPU'lar:\n" + "\n".join(gpu_list),
        instructions=f"Kullanıcı isteği: {prompt} | Önerilenler:\n" + "\n".join(gpu_list),
        agents=[agent]
    )

    return f"🎯 Önerilen GPU'lar:\n" + "\n".join(gpu_list)

# Sohbet başlat
conv = []
print("GPU Asistanı hazır! 'exit' diyene kadar devam.")
while True:
    u = input("Sen: ")
    if u.lower() == 'exit':
        break
    res = gpu_oner(u)
    conv.append({'role': 'user', 'content': u})
    conv.append({'role': 'assistant', 'content': res})
    print("Bot:", res)


GPU Asistanı hazır! 'exit' diyene kadar devam.


Sen:  profesyonel uygulamalar için çok yüksek performansa ihtiyacım var


Bot: 🎯 Önerilen GPU'lar:
RTX A6000 (VRAM: 48GB, CUDA: 10752.0, Performans: çok yüksek)
GeForce RTX 3090 Ti (VRAM: 24GB, CUDA: 10752.0, Performans: çok yüksek)
A100-PCIE-40GB (VRAM: 40GB, CUDA: 6912.0, Performans: çok yüksek)
GeForce RTX 3090 (VRAM: 24GB, CUDA: 10496.0, Performans: çok yüksek)
GeForce RTX 3080 Ti (VRAM: 12GB, CUDA: 10240.0, Performans: çok yüksek)


Sen:  oyun için orta performansta hangi gpuları önerirsin 


Bot: 🎯 Önerilen GPU'lar:
GeForce RTX 3060 (VRAM: 12GB, CUDA: 3584.0, Performans: orta)
GeForce RTX 4060 (VRAM: 8GB, CUDA: 3072.0, Performans: orta)
GeForce RTX 4060 Ti (VRAM: 8GB, CUDA: 4352.0, Performans: orta)
GeForce GTX 1080 Ti (VRAM: 11GB, CUDA: 3584.0, Performans: orta)
GeForce RTX 3060 Ti (VRAM: 8GB, CUDA: 4864.0, Performans: orta)


Sen:  exit
